<h1> Tutorial 2.1 - A Bistable Genetic Switch </h1>

The [*lac*](https://en.wikipedia.org/wiki/Lac_operon) system is perhaps the most well studied genetic switch. When *E. coli* is put in lactose, a regulatory system is initiated that incudes the expression of a lactose permease (*lacY*) that brings in the sugar. In this simulation, we start with cells that are in an uninduced state (e.g. they have low copy number of *lacY*) where a repressor dimer molecule (R2) sits on the operator site of the *lac* gene, preventing it from being transcribed. At the start of the simulation, they are introduced to a concentration of lactose inducer (I; here an analog of lactose that cannot be metabolized).  Depending on the initial concentration and the time considered, a bistable population will arise with different populations of induced (high copy number of *lac* permease) and uninduced (low copy number of *lac* permease) cells. <br/>

The model presented here can be found in the article: [Noise Contributions in an Inducible Genetic Switch: A Whole-Cell Simulation Study](http://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1002010).

In [ ]:
import os
from pyLM import *
from pyLM.units import *
from pySTDLM import *
from pySTDLM.PostProcessing import *
import math as m
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
# Create our simulation object
sim=CME.CMESimulation(name="Lac Genetic Switch")

<h2> Species </h2>

<h3> Lactose Gene, Product and Inducer Species </h3>
O - *lac* operator (e.g. the *lac* gene); it is transcribed to produce mY <br/>
mY - mRNA that encodes the lactose permease; it is translated to produce Y<br/>
Y - lactose permease; it can transport inducer into the cell <br/>
I - intracellular inducer (lactose analog) <br/>
Iex - extracellular inducer <br/>
YI - lactose analog bound by the lactose permease <br/>

<h3> Free (cytosolic) Repressor Sepcies</h3>
R2 - free repressor dimer <br/>
IR2 - free repressor with a single inducer bound <br/>
I2R2 - free repressor with inducer bound to both dimer subunits <br/>

<h3> Repressed Gene Species </h3>
R2O - repressed gene with dimer sitting on the operator <br/>
IR2O - repressed gene with a single inducer bound to the dimer <br/>
I2R2O - repressed gene with inducer molecules bound to both dimer subunits <br/>


In [ ]:
# Add the reactants
species = ['R2','O','R2O','IR2','IR2O','I2R2','I2R2O','mY','Y','I','Iex','YI']
sim.defineSpecies(species)

In [ ]:
scalar = 2.076e-9 # Rate conversion from experiments to stochastic

# Add the reactions
# Lac operon regulation
sim.addReaction(reactant=('R2','O'), product='R2O',rate=2.43e6*scalar)
sim.addReaction(('IR2','O'),  'IR2O',       1.21e6*scalar)
sim.addReaction(('I2R2','O'), 'I2R2O',      2.43e4*scalar)
sim.addReaction('R2O',        ('R2','O'),   6.30e-4)
sim.addReaction('IR2O',       ('IR2','O'),  6.30e-4)
sim.addReaction('I2R2O',      ('I2R2','O'), 3.15e-1)

# Transcription, translation, and degredation
sim.addReaction('O',          ('O','mY'),   1.26e-1)
sim.addReaction('mY',         ('mY','Y'),   4.44e-2)
sim.addReaction('mY',         '',           1.11e-2)
sim.addReaction('Y',          '',           2.10e-4)

# Inducer-repressor interactions
sim.addReaction(('I','R2'),   'IR2',        2.27e4*scalar)
sim.addReaction(('I','IR2'),  'I2R2',       1.14e4*scalar)
sim.addReaction(('I','R2O'),  'IR2O',       6.67e2*scalar)
sim.addReaction(('I','IR2O'), 'I2R2O',      3.33e2*scalar)
sim.addReaction('IR2',        ('I','R2'),   2.00e-1)
sim.addReaction('I2R2',       ('I','IR2'),  4.00e-1)
sim.addReaction('IR2O',       ('I','R2O'),  1.00)
sim.addReaction('I2R2O',      ('I','IR2O'), 2.00)

# Inducer transport
sim.addReaction('Iex',        'I',          2.33e-3)
sim.addReaction('I',          'Iex',        2.33e-3)
sim.addReaction(('Y','Iex'),  ('YI','Iex'), 3.03e4*scalar)
sim.addReaction('YI',         ('Y','Iex'),  1.20e-1)
sim.addReaction('YI',         ('Y','I'),    1.20e+1)


<h2> Initial Conditions </h2>

We start in an undiced state with no repressors bound to the operators. <br/>

We start with a concentration of about 27 $\mu$M (assuming a cell is about 1.8 fL). You may try investigating how different concentrations effect the switching time. We suggest trying 0, 8, 16, 24, 32 and 40 $\mu$M and investigating the switching time over 10 hours.

In [ ]:
# Populate the model with particles
sim.addParticles(species='R2',   count=9)
sim.addParticles('O',    1)
sim.addParticles('Y',    30)
sim.addParticles('I',    0)
sim.addParticles('Iex',  7224*4)

In [ ]:
# Set up the times
sim.setTimestep(ms(100))
sim.setWriteInterval(1)
sim.setSimulationTime(3600.0)

In [ ]:
# Save the simulation state to a file
filename='T2.1-lac2state.lm'
os.system("rm -rf %s"%(filename))
sim.save(filename)

In [ ]:
# Interrogate the Simulation
sim

In [ ]:
# Run the simulation for 10 hours
reps=10
sim.run(filename, "lm::cme::GillespieDSolver", reps)

In [ ]:
# Post-processing
fileHandle=openLMFile(filename)
for i in range(1,min(reps,10)):
    plotTraceFromFile(filename, ['mY','Y'], i, "Trace.mY.Y.%d.png"%(i))
    plotTrace(fileHandle, ["mY","Y"],i)
    plt.show()
closeLMFile(fileHandle)

<h2> Custom Post-Processing </h2>

The following code shows how to do some custom post-processing.  We pull out the species traces for the mRNA and the protein for the *lac* permease and then compute a histogram of values encountered in the simulation. You may see the bimodality when you run for 10 hours at intermediate concentrations.  

In [ ]:
# Get out the data
fileHandle=openLMFile(filename)
times = getTimesteps(fileHandle)
simmY = reps*[]
simY  = reps*[]
for i in range(1,reps):
    simY.append(getSpecieTrace(fileHandle, 'Y', i))
    simmY.append(getSpecieTrace(fileHandle, 'mY', i))

# Find min and max of mY and Y
minY  = 1e9; maxY  = 0; minmY = 1e9; maxmY = 0
for i in range(reps-1):
    for y in simY[i]:
        minY = min(minY, y)
        maxY = max(maxY, y)
    for my in simmY[i]:
        minmY = min(minmY, my)
        maxmY = max(maxmY, my)

# Histogram the data (the hard way)
hist = np.zeros(shape=(maxmY,maxY))
for i in range(reps-1):
    for j in range(len(times)):
        hist[simmY[i][j]-1, simY[i][j]-1] += 1.0
# Compute logarithm of histogram value
histLog = np.zeros(shape=(maxmY,maxY))
for i in range(maxmY-1):
    for j in range(maxY-1):
        if hist[i,j] > 0:
            histLog[i,j] = m.log10(hist[i,j])
            
# Plot ourselves a histogram
plt.clf()
plt.imshow(histLog.transpose(), interpolation='nearest', origin='lower', aspect='auto')
plt.colorbar()
plt.xlabel('mRNA Count')
plt.ylabel('LacY Count')
plt.savefig('LacYmRNAHeatmap.png')

# Clean up after ourselves
closeLMFile(fileHandle)


<h2> Assignment 2 </h2>

1. Try examining the switching time using different concentrations.  What bimodality (of species Y) do you get using 0, 8, 16, 24 or 32 $\mu$M inducer concentrations?

2. How long bimodality depend on the time of incubation in inducer? (Hint: run the system for 10-20 hours at different concentrations)

3. (Challenge) Try plotting the the bimodality seen in the last timestep as a histogram. You may find Matplotlib's [pyplot.hist()](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.hist) to be useful for this. (Hint: you will need to increase the number of simulation replicates to get smooth histograms).